In [ ]:
"""
Extração dos dados
"""

In [4]:
import pandas as pd
import os
import kagglehub
from sqlalchemy import create_engine
from dotenv import load_dotenv
import datetime

C:\Users\hamms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path = kagglehub.dataset_download("docstein/brics-world-bank-indicators")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\hamms\.cache\kagglehub\datasets\docstein\brics-world-bank-indicators\versions\2


In [ ]:
"""
Análise exploratória
"""

In [6]:
#Nome dos arquivos baixados 
print(os.listdir(path))

['BRICS Development Indicators', 'BRICS Economic Data.xlsx', 'BRICS Education and Environment Data.xlsx', 'BRICS Health and Poverty Data.xlsx', 'BRICS Private Sector Data.xlsx', 'BRICS Public Sector and Society Data.xlsx']


In [7]:
Arquivos = ['BRICS Economic Data.xlsx', 'BRICS Education and Environment Data.xlsx', 'BRICS Health and Poverty Data.xlsx',
            'BRICS Private Sector Data.xlsx', 'BRICS Public Sector and Society Data.xlsx']

for arquivo in Arquivos:
    print(f"\n{'='*50}\n Analisando: {arquivo}\n{'='*50}")
    df = pd.read_excel(os.path.join(path, arquivo))

    print("\n Info geral:")
    print(df.info())

    print("\n Tipos de dados:")
    print(df.dtypes)

    print("\n Primeiras linhas:")
    print(df.head())

    print("\n Valores nulos por coluna:")
    print(df.isnull().sum())

    print("\n Strings vazias por coluna:")
    print((df == '').sum())

    print("\n Registros duplicados:", df.duplicated().sum())

    print("\n Valores únicos por coluna:")
    print(df.nunique())

    print("\n📊 Estatísticas descritivas:")
    print(df.describe(include='all'))


 Analisando: BRICS Economic Data.xlsx

 Info geral:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 55 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   1727 non-null   object 
 1   Country Code   1725 non-null   object 
 2   Series Name    1725 non-null   object 
 3   Series Code    1725 non-null   object 
 4   1970 [YR1970]  671 non-null    float64
 5   1971 [YR1971]  688 non-null    float64
 6   1972 [YR1972]  694 non-null    float64
 7   1973 [YR1973]  693 non-null    float64
 8   1974 [YR1974]  698 non-null    float64
 9   1975 [YR1975]  802 non-null    float64
 10  1976 [YR1976]  802 non-null    float64
 11  1977 [YR1977]  808 non-null    float64
 12  1978 [YR1978]  807 non-null    float64
 13  1979 [YR1979]  835 non-null    float64
 14  1980 [YR1980]  847 non-null    float64
 15  1981 [YR1981]  882 non-null    float64
 16  1982 [YR1982]  948 non-null    float64
 17 

In [8]:
df_BRICS_Economic = pd.read_excel(os.path.join(path, Arquivos[0]))
df_BRICS_Education_and_Environment = pd.read_excel(os.path.join(path, Arquivos[1]))
df_BRICS_Health_and_Poverty = pd.read_excel(os.path.join(path, Arquivos[2]))
df_BRICS_Private_Sector = pd.read_excel(os.path.join(path, Arquivos[3]))
df_BRICS_Public_Sector_and_Society = pd.read_excel(os.path.join(path, Arquivos[4]))


In [ ]:
"""
Mudança no nome das colunas dos anos
"""

In [11]:
dfs = [
    df_BRICS_Economic,
    df_BRICS_Education_and_Environment,
    df_BRICS_Health_and_Poverty,
    df_BRICS_Private_Sector,
    df_BRICS_Public_Sector_and_Society
]

for i, df in enumerate(dfs):
    df.columns = [col.split(' ')[0] if '[YR' in col else col for col in df.columns]


In [ ]:
"""
Enviar arquivos pra SQL Server
"""

In [12]:
load_dotenv()
server = os.getenv("SERVER")
database = os.getenv("DATABASE") 

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [13]:
df_BRICS_Economic.to_sql('Economia', engine, if_exists='replace', index=False)
df_BRICS_Education_and_Environment.to_sql('Educacao_e_Ambiente', engine, if_exists='replace', index=False)
df_BRICS_Health_and_Poverty.to_sql('Saude_e_Pobreza', engine, if_exists='replace', index=False)
df_BRICS_Private_Sector.to_sql('Setor_Privado', engine, if_exists='replace', index=False)
df_BRICS_Public_Sector_and_Society.to_sql('Setor_Publico_e_Sociedade', engine, if_exists='replace', index=False)


4